 코드는 GitHub aima-python의 코드를 기반으로 일부 수정한 것임.

In [1]:
# 탐색을 통한 문제 해결을 위해 필요한 기반 구조들은 search_common.py에 코드를 옮겨서 저장해뒀음.
from search_common import *

## AND-OR 탐색
비결정적 환경에서 조건부 계획(전략) 형태의 해결책을 찾는 탐색.

비결정적 문제의 전이 모델은 결과 상태 하나가 아니라 가능한 결과 상태들의 집합을 리턴함.

해결책은 다음 두 가지 요소로 구성됨.
- 행동 시퀀스(리스트): 예: `['forward', 'suck']`
- 조건별 행동(딕셔너리): 예: `{5: ['backward', 'suck'], 7: []}`

In [2]:
def and_or_search(problem):
    """비결정적 행동을 갖는 문제에 대해 조건부 계획을 찾아냄."""
    return or_search(problem, problem.initial, [])

def or_search(problem, state, path):
    """에이전트 자신이 행동을 선택하여 분기.
    path에 있는 상태들을 반복하지 않으면서 state로부터 목표에 도달하는 행동 시퀀스 탐색."""
    if problem.is_goal(state): return []
    if state in path: return failure  # 루프 체크
    for action in problem.actions(state):
        plan = and_search(problem, problem.results(state, action), [state] + path)
        if plan != failure:
            return [action] + plan
    return failure

def and_search(problem, states, path):
    """각 행동의 결과를 환경이 선택하여 분기.
    처할 수 있는 모든 가능한 상태들에 대한 계획 리턴."""
    if len(states) == 1: 
        return or_search(problem, next(iter(states)), path)
    plan = {}
    for s in states:
        plan[s] = or_search(problem, s, path)
        if plan[s] == failure: return failure
    return [plan]

## 비결정적 문제 해결: 변덕스러운 청소 로봇

In [3]:
class MultiGoalProblem(Problem):
    """목표가 여러 개인 문제 정의 클래스."""
    def __init__(self, initial=None, goals=(), **kwds): 
        self.__dict__.update(initial=initial, goals=goals, **kwds)

    def is_goal(self, state): return state in self.goals


class ErraticVacuum(MultiGoalProblem):
    """2개의 타일 공간에서 변덕스럽게 행동하는 청소 로봇 문제.
    지저분한 타일에서 흡입 행동 -> 현재 위치 청소 or 두 위치 모두 청소.
    깨끗한 타일에서 흡입 행동 -> 아무 일도 일어나지 않음 or 현재 위치에 먼지를 쏟아냄.
    이동 행동은 결정적임.
    상태는 교재 그림과 같이 8개의 번호로 표시함."""
    
    def actions(self, state): 
        return ['suck', 'left', 'right']
    
    def results(self, state, action): return self.table[action][state]
    
    table = {'suck': {1:{5,7}, 2:{4,8}, 3:{7}, 4:{2,4}, 5:{1,5}, 6:{8}, 7:{3,7}, 8:{6,8}},
             'left': {1:{1}, 2:{1}, 3:{3}, 4:{3}, 5:{5}, 6:{5}, 7:{7}, 8:{7}},
             'right': {1:{2}, 2:{2}, 3:{4}, 4:{4}, 5:{6}, 6:{6}, 7:{8}, 8:{8}}}

In [4]:
and_or_search(ErraticVacuum(1, {7, 8}))

['suck', {5: ['right', 'suck'], 7: []}]

In [5]:
# 초기 상태별 탐색 결과
{s: and_or_search(ErraticVacuum(s, {7,8})) for s in range(1, 9)}

{1: ['suck', {5: ['right', 'suck'], 7: []}],
 2: ['suck', {8: [], 4: ['left', 'suck']}],
 3: ['suck'],
 4: ['left', 'suck'],
 5: ['right', 'suck'],
 6: ['suck'],
 7: [],
 8: []}